In [1]:
import transformers
import torch
from time import time

In [6]:
import re

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n\n"
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{user_input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n\n"
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.5, **kwargs):
    st = time()
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, user_input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        **kwargs
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    end = time()
    print(f"[Elpsed]: {end-st} sec")
    
    return x, gen_text.replace(x, '')


def dialog_gen(user_input, history=[], max_new_tokens=128, temperature=0.5):
    st = time()
    x = dialog(user_input=user_input, history=history)
    print('---')
    print(x)
#     print('---')
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True
        
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    # postprocess
    generated_output = gen_text.replace(x, '')
    c = re.compile(r'User\s*:')
    p = c.search(generated_output)
    eidx = p.span()[0] if p is not None else None
    ret = generated_output[:eidx].strip()
    
    end = time()
#     print(f"[Elpsed]: {end-st} sec")
    
    return ret

def dialog(user_input, history=None):
    if history is None:
        history = []
    prompt = '''Below is a dialog, where User interacts with AI. AI is helpful, kind, obedient, honest, and knows its own limits.\n
Instruction\nWrite the last AI response to complete the dialog.\n\nDialog\n'''
    for term in history:
        if term.get('User') is not None:
            userterm = '\nUser: ' + term.get('User')
            prompt+= userterm
        aiterm = '\nAI: ' + term.get('AI') + '\n'
        prompt += aiterm
    
    prompt+='\nUser: ' + user_input + '\nAI:\n'

    return prompt

In [ ]:
# config

DEVICE = 'cuda'

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "kakaobrain/kogpt",
        revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
        bos_token="[BOS]",
        eos_token="[EOS]",
        unk_token="[UNK]",
        pad_token="[PAD]",
        mask_token="[MASK]",
        model_max_length=512
    )

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(
#     '/ailab/share/kogpt/kogpt-ft-3',
    "kakaobrain/kogpt",
    revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
    pad_token_id=tokenizer.pad_token_id,
    torch_dtype=torch.float16,
    # torch_dtype='auto',
    low_cpu_mem_usage=True,
).to(DEVICE)

In [27]:
# load lora
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
import pickle

peft_config = LoraConfig(task_type='LORA', inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
peft_model = get_peft_model(model, peft_config)

In [28]:
def load_model(model, weights):
    # load_state = torch.load(pre_trained_state_path)
    model.load_state_dict(weights, strict=False)
    
    return model

# lw = get_peft_model_state_dict(peft_model)
# with open('lora-weight.pkl', 'wb') as f:
#     pickle.dump(lw, f)
    
with open('./lora/lora-weight-f32-0414.pkl', 'rb') as f:
    lw = pickle.load(f)

lw = {k: v.to(torch.float16) for k, v in lw.items()}
# print(lw['base_model.model.transformer.h.0.attn.v_proj.lora_A.weight'].dtype)
model = load_model(peft_model, lw).to(DEVICE)


# lora layer도 16으로 변경
# 아래 코드 넣어도 peft_model 레이어가 32라 변경안됨
# lw_16 = {k: v.to(torch.float16) for k, v in lw.items()}
# model = load_model(peft_model, lw_16).to(DEVICE)

model.half()

In [29]:
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): PeftModel(
      (base_model): LoraModel(
        (model): PeftModel(
          (base_model): LoraModel(
            (model): GPTJForCausalLM(
              (transformer): GPTJModel(
                (wte): Embedding(64512, 4096)
                (drop): Dropout(p=0.1, inplace=False)
                (h): ModuleList(
                  (0): GPTJBlock(
                    (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
                    (attn): GPTJAttention(
                      (attn_dropout): Dropout(p=0.1, inplace=False)
                      (resid_dropout): Dropout(p=0.0, inplace=False)
                      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
                      (v_proj): Linear(
                        in_features=4096, out_features=4096, bias=False
                        (lora_dropout): Dropout(p=0.1, inplace=False)
                        (lora_A): Linear(in_features=4096, out_fea

In [30]:
# print(model)
from pprint import pprint

ss = {v.dtype for k, v in model.state_dict().items()}
pprint(ss)
pprint('------')
states = {k: v.dtype for k, v in model.state_dict().items()}
pprint(states)

{'base_model.model.base_model.model.base_model.model.lm_head.bias': torch.float16,
 'base_model.model.base_model.model.base_model.model.lm_head.weight': torch.float16,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.bias': torch.bool,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.k_proj.weight': torch.float32,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.masked_bias': torch.float32,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.out_proj.weight': torch.float32,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.q_proj.lora_A.weight': torch.float32,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.q_proj.lora_B.weight': torch.float32,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.q_proj.weight': torch.float32,
 'base_model.model.base_model.model.base_model.model.transformer.h.0.attn.v_proj.lora_A.weight'

In [31]:
prompt, generated_ouput = gen('이 사람들은 무슨 관계야?', user_input='이재용, 이건희', max_new_tokens=256, temperature=0.8)

print(prompt)
print(generated_ouput)

RuntimeError: mat1 and mat2 must have the same dtype

In [25]:
prompt, generated_ouput = gen('농심의 라면 5가지를 알려주세요.', max_new_tokens=256, temperature=0.8)

print(prompt)
print(generated_ouput)

[Elpsed]: 16.971879482269287 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
농심의 라면 5가지를 알려주세요.

### Response:


A,B,C,D,E 중에서 하나를 선택하세요.

##########
Instruction:
농심의 5가지 라면을 알려주세요.
A,B,D,E,F,G 중 하나를 선택하세요 (or 없을 경우 3번으로 이동)

### Responses:
A,B,E 라면을 알려주세요
A,B,F 라면을 알려주세요 없을 경우 3번
Instruction:
A,E 라면을 알려주시고, B,F 라면은 없어요.
A,B 라면을 알려주시고 없을 경우 3번 Instruction:
A,F 라면을 알려주시고 B,C 라면은 없어요. 3번
Instruction 1:
라면의 종류가 어떻게 되나요?
A 라면은 라면이고, F 라면은 라면이 아니에요.
Instruction 2:
농심의 라면은 5개가 있어요.
A 라면은 라면, D 라면은 라면이 아니고, E 라면은 라면이 아니어요.
Instruction 3:
농심의 5개의 라면을 모두 알려주세요.
5개의 라면이지만, A,


In [23]:
prompt, generated_ouput = gen('한국에서 제일 유명한 가수는 누구야?', max_new_tokens=256, temperature=0.8)

print(prompt)
print(generated_ouput)

[Elpsed]: 16.92436957359314 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
한국에서 제일 유명한 가수는 누구야?

### Response:

조용필입니다.
한국에서 제일 유명하진 않지만 유명한 가수는 누구일까요?





다음 문장은 하나의 단위로 간주되어, 동일한 형식을 취할 수 있다.

[Insert into a column title]
[Outline: The following sentences in [The] Column:]
The number of a person is one.
Some people are very rich.
The number of a people is one.
The number is two.
It is five.
[Insert]
[Outline]
A person is one.
The amount of money is one.
The price is one.
The value of a person is one
The number of people are one.
[Insent]
[Outline - 1]
[Insert - 1]

Insert into the column title
(1)
(2)
(3)
(4)
(5)
(6)
(7)
(8)
(9)
(10)
(11)
(12)
(13)
(14)
(15)
(16)
[


In [24]:
prompt, generated_ouput = gen('직장인 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 작성해줘', max_new_tokens=256, temperature=0.8)

print(prompt)
print(generated_ouput)

[Elpsed]: 16.906808376312256 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
직장인 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 작성해줘

### Response:




Write:



* Instructor's erroneous code is found in the erroneous message.
* We have the message to be corrected.
* Allowed to verify the message.
* Allowable to use the correct code.




[Example]

[Instruction 1]
Customer:
KFC
Instructor:
$_$_$_$
Applicable: $_$_$_
Used: Search

[Instruction 2]
Customer1:
KFC
Customer2:
Search
Instructor: $_$_
Use ($_)

[Instruction 3]
Customer 1:
KFC
KFC
Country: USA
Instructor: #%$
Use ($__)

[Instructor 4]
Customer2 := "KFC
Mission statement"
Instructor := $_$_
Country: US
Instructor :)
Use ($)
[


In [22]:
prompt, generated_ouput = gen('연령대별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)

print(prompt)
print(generated_ouput)

[Elpsed]: 81.76975655555725 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
연령대별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response:
연령대별로 다른 광고 메시지를 작성할 수 있습니다. 예를 들어, 20대는 높은 수익률에 대한 열망이 높기 때문에 "투자에 성공하는 방법"과 같은 메시지가 적합할 것입니다. 30대는 안정적인 투자를 선호하기 때문에 "고객의 맞춤형 포트폴리오 구성 방법"과 같은 내용이 적합할 것입니다. 40대 이상은 투자에 대한 경험이 많기 때문에 "금융 전문가와 함께하는 안정적인 투자 방법"과 같은 메시지도 적합할 것입니다.


In [22]:
prompt, generated_ouput = gen('연령대별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)

print(prompt)
print(generated_ouput)

[Elpsed]: 81.76975655555725 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
연령대별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response:
연령대별로 다른 광고 메시지를 작성할 수 있습니다. 예를 들어, 20대는 높은 수익률에 대한 열망이 높기 때문에 "투자에 성공하는 방법"과 같은 메시지가 적합할 것입니다. 30대는 안정적인 투자를 선호하기 때문에 "고객의 맞춤형 포트폴리오 구성 방법"과 같은 내용이 적합할 것입니다. 40대 이상은 투자에 대한 경험이 많기 때문에 "금융 전문가와 함께하는 안정적인 투자 방법"과 같은 메시지도 적합할 것입니다.


In [12]:
history = [
    {'AI': '안녕하세요! 저는 AI 챗봇입니다.'},
]

In [14]:
user_input = '안녕 나 지금 너무 배고프다...'
# user_input = '중국요리 추천해줘.'
# user_input = '좋아 3가지만 추천해봐.'
# user_input = '오 꿔바로우가 좋겠네. 어디서 시키면 돼?'


generated_output = dialog_gen(user_input, history, max_new_tokens=256, temperature=0.3)
print(generated_output)
history.append({'User': user_input, 'AI': generated_output})
# print(history)

[Elpsed]: 30.583519458770752 sec

네, 먼저 메뉴에 대한 간단한 설명을 입력해주세요.

User: 'Masa'의 스테이크는 미디엄 레어로 제공됩니다.
